In [1]:
#coding = utf-8
import pandas as pd
import numpy as np
from operator import itemgetter
import math 
import json
import pickle
import re
import os
import csv
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfVectorizer
import logging
import jieba.posseg as pseg

In [2]:
root = r'C:\Users\Evilemon\Desktop\recommender\exam\qunar_jd_comment\comment'
result = r'C:\Users\Evilemon\Desktop\recommender\exam\result'

In [3]:
def deleteBysize(root):           
    for dirpath, dirnames, filenames in os.walk(root):
        for filename in filenames:
            if os.path.getsize(os.path.join(dirpath, filename)) < 1 * 1024:
                os.remove(os.path.join(dirpath, filename))
                print(filename + 'deleted')
    return 0

In [4]:
deleteBysize(root)

0

In [5]:
def ItemUserMatrix(root,result):
    user_dict = dict()              # user_id 的数据字典
    jd_dict = dict()                #jd_poi 的数据字典
    jd_users_dict = dict()          #每个jd 对应的 去旅游过并进行评论过的用户的id
    user_jds_dict = dict()          #每个user去过并评论的jd的poi
    n = 0                           # user_id 的数据字典中的数字排序
    m = 0                           #jd_poi 的数据字典中的数字排序
    mat = np.full((174718,2687),0.65)   #每一行代表用户对几个景点的评分数据
    for dirpath, dirnames, filenames in os.walk(root):
        for filename in filenames:
            rpath = os.path.join(dirpath, filename)
            with open(rpath, 'rb') as cf:
                df = pd.DataFrame(pd.read_csv(cf, encoding="unicode_escape", lineterminator= '\n'))
            M = df.as_matrix()      #将Dataframe数据转换为矩阵形式
            jd = str(M[0][3])     #将每个景点的jd_poi提取出来并转换成字符串
            x = 0                 #代表对每个不同的数据文件时都要从0行开始读取
            #jd的数据字典赋值
            if jd not in jd_dict.keys():   
                jd_dict[jd] = m
                
            #创建jd_users_dict字典
            if jd not in jd_users_dict.keys():
                jd_users_dict[m] = set()
            #读取user_id并传换成字符串，同时向jd_users_dict中添加用户
            for I in M[:,2]:
                
                if I not in user_jds_dict.keys():
                    user_jds_dict[n] = set()
               
                
                #user数据字典赋值，并且对mat赋值
                if I not in user_dict.keys():
                    user_dict[I] = n
                    jd_users_dict[m].add(n)
                    user_jds_dict[n].add(m)
                    a = 0
                    a = M[x][4]
                    b = a/5         #用最大值对评分做个正则化
                    mat[n][m] = b
                    n += 1
                    x += 1
                else:
                    c = user_dict[I]
                    jd_users_dict[m].add(c)
                    user_jds_dict[c].add(m)
                    a = 0
                    a = M[x][4]
                    b = a/5         #用最大值对评分做个正则化
                    mat[c][m] = b
                    x += 1
            m += 1
    #将得到的数据写入文件
    print(n)
    print(m)
    user_dict_path = os.path.join(result, 'user_dict')
    jd_dict_path = os.path.join(result, 'jd_dict')
    jd_users_dict_path = os.path.join(result, 'jd_users_dict')
    user_jds_dict_path = os.path.join(result, 'user_jds_dict')
    ItemUserMatrix_path = os.path.join(result, 'ItemUserMatrix')
    with open(user_dict_path, 'wb') as uf:
        pickle.dump(user_dict, uf)
    with open(jd_dict_path, 'wb') as jf:
        pickle.dump(jd_dict, jf)
    with open(jd_users_dict_path, 'wb') as juf:
        pickle.dump(jd_users_dict, juf)
    with open(user_jds_dict_path, 'wb') as ujf:
        pickle.dump(user_jds_dict, ujf)
    np.save(ItemUserMatrix_path,mat)
    return user_dict,jd_dict,jd_users_dict,user_jds_dict, mat
i,j,x,y,z = ItemUserMatrix(root,result)

174718
2687


In [10]:
def ItemSimilarity(user_jds_dict, result):
    #caculate co-rated users between jds
    N = dict()
    C = np.zeros((2687,2687))
    for user, jds in user_jds_dict.items():
        for i in jds:
            if i not in N:
                N[i] = 0
            N[i] += 1
            for j in jds:
                if i == j:
                    continue
                C[i][j] += 1 / math.log(1 + len(jds))
   
    #cacualte finial similarity matrix W
    W = np.zeros((2687, 2687))
    for i in range(len(C)):
        for j in range(len(C[0])):
            W[i][j] = C[i][j] / math.sqrt(N[i] * N[j])
    jd_users_num_dict = N           #每个jd有多少人去过的字典
    jd_users_num_matrix = C         #矩阵中每一个元素代表俩个 jd 被多少个用户都去过
    jd_similarity_matrix = W        #矩阵中每一个元素代表两个 jd 的相似度
    #将数据写入文件
    jd_users_num_dict_path = os.path.join(result, 'jd_users_num_dict')
    jd_users_num_matrix_path = os.path.join(result, 'jd_users_num_matrix')
    jd_similarity_matrix_path = os.path.join(result, 'jd_similarity_matrix')
    with open(jd_users_num_dict_path, 'wb') as ujnf:
        pickle.dump(jd_users_num_dict, ujnf)
    np.save(jd_users_num_matrix_path, jd_users_num_matrix)
    np.save(jd_similarity_matrix_path, jd_similarity_matrix)
    for k in sorted(N):
        print(k,N[k])
    return W
W = ItemSimilarity(y, result)
#print(W)

0 15
1 28
2 10
3 9
4 34
5 5
6 13
7 23
8 11
9 4
10 2
11 17
12 4
13 8
14 6
15 8
16 158
17 37
18 4
19 160
20 4
21 6
22 9
23 3
24 52
25 163
26 6
27 17
28 22
29 28
30 16
31 14
32 15
33 20
34 2
35 19
36 10
37 132
38 55
39 17
40 7
41 99
42 20
43 250
44 24
45 16
46 12
47 3
48 31
49 34
50 2
51 13
52 73
53 152
54 19
55 8
56 15
57 109
58 23
59 59
60 14
61 27
62 5
63 29
64 25
65 25
66 27
67 7
68 30
69 64
70 55
71 63
72 30
73 2
74 9
75 10
76 17
77 42
78 48
79 96
80 21
81 121
82 90
83 42
84 47
85 46
86 267
87 137
88 17
89 239
90 51
91 920
92 773
93 302
94 161
95 656
96 353
97 2
98 246
99 1241
100 937
101 285
102 403
103 90
104 68
105 126
106 114
107 53
108 158
109 171
110 60
111 156
112 8
113 56
114 129
115 46
116 48
117 15
118 134
119 9
120 485
121 29
122 24
123 6
124 13
125 30
126 6
127 24
128 139
129 50
130 1565
131 9
132 29
133 50
134 171
135 90
136 89
137 369
138 97
139 16
140 23
141 64
142 18
143 193
144 15
145 16
146 734
147 184
148 69
149 89
150 27
151 84
152 410
153 216
154 177
155 132
156 

1213 35
1214 32
1215 156
1216 2225
1217 229
1218 596
1219 101
1220 507
1221 3
1222 4
1223 106
1224 284
1225 184
1226 126
1227 2
1228 8
1229 115
1230 3
1231 5
1232 64
1233 57
1234 11
1235 41
1236 96
1237 3
1238 32
1239 36
1240 5
1241 12
1242 41
1243 44
1244 55
1245 49
1246 158
1247 11
1248 50
1249 410
1250 70
1251 6
1252 129
1253 17
1254 170
1255 27
1256 223
1257 11
1258 50
1259 13
1260 4
1261 67
1262 43
1263 9
1264 10
1265 57
1266 65
1267 13
1268 31
1269 36
1270 35
1271 25
1272 74
1273 5
1274 7
1275 40
1276 56
1277 64
1278 18
1279 32
1280 72
1281 225
1282 9
1283 19
1284 86
1285 16
1286 84
1287 7
1288 279
1289 1157
1290 14
1291 99
1292 14
1293 191
1294 163
1295 9
1296 174
1297 13
1298 55
1299 200
1300 131
1301 958
1302 69
1303 91
1304 14
1305 28
1306 7
1307 144
1308 88
1309 310
1310 32
1311 87
1312 60
1313 834
1314 29
1315 116
1316 289
1317 241
1318 22
1319 229
1320 382
1321 3156
1322 551
1323 149
1324 212
1325 586
1326 210
1327 1119
1328 75
1329 419
1330 4
1331 85
1332 54
1333 31
1334 

In [26]:
def Recommendation(user_jds_dict, jd_similarity_matrix, ItemUserMatrix, user_id, K):
    rank = dict()
    ru = user_jds_dict[user_id]
    for i in ru:
        y = jd_similarity_matrix[:,i].argsort()
        for j in y[K:]:
            if j not in ru:
                continue
            if j not in rank.keys():
                rank[j] = 0
            rank[j] = jd_similarity_matrix[i][j] * ItemUserMatrix[user_id][i]
    return rank
R = Recommendation(y, W, z, 17,-10)
print(sorted(R, reverse = True))

[1840, 1]
